#Download Dataset

In [ ]:
!git clone https://github.com/TalwalkarLab/leaf.git
%cd leaf
!pip install -r requirements.txt

Cloning into 'leaf'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 782 (delta 1), reused 2 (delta 0), pack-reused 776
Receiving objects: 100% (782/782), 6.79 MiB | 4.91 MiB/s, done.
Resolving deltas: 100% (372/372), done.
/content/leaf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==1.13.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribu

In [ ]:
%cd data/shakespeare
!./preprocess.sh --sf 0.2 -t sample -tf 0.8
%cd ../..

/content/leaf/data/shakespeare
--2024-06-06 12:44:10--  http://www.gutenberg.org/files/100/old/1994-01-100.zip
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/100/old/1994-01-100.zip [following]
--2024-06-06 12:44:11--  https://www.gutenberg.org/files/100/old/1994-01-100.zip
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2035857 (1.9M) [application/zip]
Saving to: ‘1994-01-100.zip’

1994-01-100.zip     100%[===================>]   1.94M  1.28MB/s    in 1.5s    

2024-06-06 12:44:13 (1.28 MB/s) - ‘1994-01-100.zip’ saved [2035857/2035857]

Archive:  1994-01-100.zip
  inflating: 100.txt                 
dividing txt data between users
Splitting .txt data between users
Discard

#Tokenize dataset

In [ ]:
ALL_LETTERS = "\n !\"&'(),-.0123456789:;>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz}"
NUM_LETTERS = len(ALL_LETTERS)
print(NUM_LETTERS)

def _one_hot(index, size):
    '''returns one-hot vector with given size and value 1 at given index
    '''
    vec = [0 for _ in range(size)]
    vec[int(index)] = 1
    return vec

def letter_to_vec(letter):
    '''returns one-hot representation of given letter
    '''
    index = ALL_LETTERS.find(letter)
    return _one_hot(index, NUM_LETTERS)

def word_to_indices(word):
    '''returns a list of character indices

    Args:
        word: string

    Return:
        indices: int list with length len(word)
    '''
    indices = []
    for c in word:
        indices.append(ALL_LETTERS.find(c))
    return indices

80


In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import json
import os

def read_data(train_data_path, test_data_path):
    def read_file(path):
        with open(path, 'r') as f:
            return json.load(f)

    train_data = read_file(train_data_path)
    test_data = read_file(test_data_path)

    train_clients = list(train_data['users'])
    train_groups = list(train_data['user_data'].keys())
    train_data_temp = train_data['user_data']

    test_clients = list(test_data['users'])
    test_groups = list(test_data['user_data'].keys())
    test_data_temp = test_data['user_data']

    return train_clients, train_groups, train_data_temp, test_data_temp

class ShakeSpeare(Dataset):
    def __init__(self, train=True, args=None):
        super(ShakeSpeare, self).__init__()
        train_clients, train_groups, train_data_temp, test_data_temp = read_data(args.shakespeare_train_path, args.shakespeare_test_path)
        self.train = train

        if self.train:
            self.dic_users = {}
            train_data_x = []
            train_data_y = []
            for i in range(len(train_clients)):
                self.dic_users[i] = set()
                l = len(train_data_x)
                cur_x = train_data_temp[train_clients[i]]['x']
                cur_y = train_data_temp[train_clients[i]]['y']
                for j in range(len(cur_x)):
                    self.dic_users[i].add(j + l)
                    train_data_x.append(cur_x[j])
                    train_data_y.append(cur_y[j])
            self.data = train_data_x
            self.label = train_data_y
        else:
            test_data_x = []
            test_data_y = []
            for i in range(len(train_clients)):
                cur_x = test_data_temp[train_clients[i]]['x']
                cur_y = test_data_temp[train_clients[i]]['y']
                for j in range(len(cur_x)):
                    test_data_x.append(cur_x[j])
                    test_data_y.append(cur_y[j])
            self.data = test_data_x
            self.label = test_data_y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence, target = self.data[index], self.label[index]
        indices = word_to_indices(sentence)
        target = letter_to_vec(target)
        indices = torch.LongTensor(np.array(indices))
        target = torch.FloatTensor(np.array(target))
        return indices, target

    def get_client_dic(self):
        if self.train:
            return self.dic_users
        else:
            raise ValueError("The test dataset does not have dic_users!")

#Model

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class CharLSTM(nn.Module):
    def __init__(self):
        super(CharLSTM, self).__init__()

        embedding_dim = 8
        hidden_size = 100
        num_LSTM = 2
        input_length = 80
        self.n_cls = 80

        self.embedding = nn.Embedding(input_length, embedding_dim)
        self.stacked_LSTM = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_LSTM)
        self.fc = nn.Linear(hidden_size, self.n_cls)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2) # lstm accepts in this style
        output, (h_, c_) = self.stacked_LSTM(x)
        # Choose last hidden layer
        last_hidden = output[-1,:,:]
        x = self.fc(last_hidden)

        return x

#Training parameter

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

# Define the model, loss function, and optimizer
model = CharLSTM()
criterion = nn.CrossEntropyLoss()

# Ciclo di addestramento
epochs = 10

# Define the optimizer (Stochastic Gradient Descent with Momentum)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

In [ ]:
class Args:
    shakespeare_train_path = '/content/leaf/data/shakespeare/data/train/all_data_train_9.json'
    shakespeare_test_path = '/content/leaf/data/shakespeare/data/test/all_data_test_9.json'

args = Args()
train_dataset = ShakeSpeare(train=True, args=args)
test_dataset = ShakeSpeare(train=False, args=args)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#Training and Plot

##Definition test and test

In [ ]:
import matplotlib.pyplot as plt

def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backward pass
        loss.backward()

        # Optimize the weights
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 99:  # Print every 100 batches
            #print(f'Batch {batch_idx + 1}, Loss: {running_loss / 100:.6f}')
            running_loss = 0.0

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = criterion(output, target)
            test_loss += loss.item()

            # Get the index of the max log-probability
            _, predicted = torch.max(output, 1)
            _, target_idx = torch.max(target, 1)
            total += target.size(0)
            correct += (predicted == target_idx).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / total
    print(f'Test Loss: {test_loss:.6f}, Accuracy: {correct}/{total} ({accuracy:.2f}%)')

    return test_loss, accuracy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CharLSTM(
  (embedding): Embedding(80, 8)
  (stacked_LSTM): LSTM(8, 100, num_layers=2)
  (fc): Linear(in_features=100, out_features=80, bias=True)
)

##Training and plot

In [ ]:
import matplotlib.pyplot as plt

# Initialize lists to store test loss and accuracy
test_losses = []
test_accuracies = []

# Training settings
epochs = 25 #20

for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    train(model, train_loader, criterion, optimizer, device)
    test_loss, accuracy = test(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)

# Plot test loss
plt.plot(range(1, epochs + 1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Test Loss over Epochs')
plt.legend()
plt.savefig('test_loss_plot.pdf')
plt.close()

# Plot test accuracy
plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy over Epochs')
plt.legend()
plt.savefig('test_accuracy_plot.pdf')
plt.close()

Epoch 1/25
Test Loss: 0.026906, Accuracy: 176968/356921 (49.58%)
Epoch 2/25
Test Loss: 0.025335, Accuracy: 185714/356921 (52.03%)
Epoch 3/25
Test Loss: 0.024805, Accuracy: 188939/356921 (52.94%)
Epoch 4/25
Test Loss: 0.024265, Accuracy: 191702/356921 (53.71%)
Epoch 5/25
Test Loss: 0.023991, Accuracy: 192863/356921 (54.04%)
Epoch 6/25
Test Loss: 0.023848, Accuracy: 194146/356921 (54.39%)
Epoch 7/25
Test Loss: 0.023724, Accuracy: 194107/356921 (54.38%)
Epoch 8/25
Test Loss: 0.023688, Accuracy: 194589/356921 (54.52%)
Epoch 9/25
Test Loss: 0.023562, Accuracy: 195374/356921 (54.74%)
Epoch 10/25
Test Loss: 0.023396, Accuracy: 196232/356921 (54.98%)
Epoch 11/25
Test Loss: 0.023354, Accuracy: 196392/356921 (55.02%)
Epoch 12/25
Test Loss: 0.023371, Accuracy: 196197/356921 (54.97%)
Epoch 13/25
Test Loss: 0.023314, Accuracy: 196950/356921 (55.18%)
Epoch 14/25
Test Loss: 0.023330, Accuracy: 197011/356921 (55.20%)
Epoch 15/25
Test Loss: 0.023295, Accuracy: 196451/356921 (55.04%)
Epoch 16/25
Test Lo

# Hyperparameter Tuning

## lr e wd

In [ ]:
epochs = 10

# Define a function for model training and testing
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device):
    model.to(device)
    test_losses = []
    test_accuracies = []

    for epoch in range(1, epochs + 1):
        print(f'Epoch {epoch}/{epochs}')
        train(model, train_loader, criterion, optimizer, device)
        test_loss, accuracy = test(model, test_loader, criterion, device)
        test_losses.append(test_loss)
        test_accuracies.append(accuracy)

    return test_losses, test_accuracies

# Define hyperparameters and ranges for LR and WD
learning_rates = [0.001, 0.01, 0.1]
weight_decays = [0.0001, 0.001, 0.01]

best_lr = None
best_wd = None
best_accuracy = 0.0

# Iterate over all combinations of LR and WD
for lr in learning_rates:
    for wd in weight_decays:
        # Define your model, optimizer, and criterion
        model = CharLSTM()  # Replace YourModel with your actual model class
        optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=lr, weight_decay=wd)
        criterion = nn.CrossEntropyLoss()

        # Train and test the model
        test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device)

        # Get the final accuracy
        final_accuracy = test_accuracies[-1]

        # Check if this combination gives the best accuracy so far
        if final_accuracy > best_accuracy:
            best_lr = lr
            best_wd = wd
            best_accuracy = final_accuracy

# Print the best LR, WD, and accuracy
print(f'Best LR: {best_lr}, Best WD: {best_wd}, Best Accuracy: {best_accuracy:.2f}%')

Epoch 1/10
Test Loss: 0.039672, Accuracy: 109243/356921 (30.61%)
Epoch 2/10
Test Loss: 0.034310, Accuracy: 138947/356921 (38.93%)
Epoch 3/10
Test Loss: 0.031625, Accuracy: 152499/356921 (42.73%)
Epoch 4/10
Test Loss: 0.029899, Accuracy: 162285/356921 (45.47%)
Epoch 5/10
Test Loss: 0.028831, Accuracy: 168456/356921 (47.20%)
Epoch 6/10
Test Loss: 0.028122, Accuracy: 171772/356921 (48.13%)
Epoch 7/10
Test Loss: 0.027478, Accuracy: 174779/356921 (48.97%)
Epoch 8/10
Test Loss: 0.027100, Accuracy: 176878/356921 (49.56%)
Epoch 9/10
Test Loss: 0.026718, Accuracy: 178798/356921 (50.09%)
Epoch 10/10
Test Loss: 0.026420, Accuracy: 180007/356921 (50.43%)
Epoch 1/10
Test Loss: 0.045084, Accuracy: 87337/356921 (24.47%)
Epoch 2/10
Test Loss: 0.038091, Accuracy: 116294/356921 (32.58%)
Epoch 3/10
Test Loss: 0.036379, Accuracy: 131086/356921 (36.73%)
Epoch 4/10
Test Loss: 0.035041, Accuracy: 135969/356921 (38.09%)
Epoch 5/10
Test Loss: 0.034169, Accuracy: 140854/356921 (39.46%)
Epoch 6/10
Test Loss: 0.0

##With Scheduler

In [ ]:
import matplotlib.pyplot as plt


# Initialize lists to store test loss and accuracy
test_losses = []
test_accuracies = []

# Training settings

for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    train(model, train_loader, criterion, optimizer, device)
    test_loss, accuracy = test(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)

# Plot test loss
plt.plot(range(1, epochs + 1), test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Test Loss over Epochs')
plt.legend()
plt.savefig('test_loss_plot.pdf')
plt.close()

# Plot test accuracy
plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy over Epochs')
plt.legend()
plt.savefig('test_accuracy_plot.pdf')
plt.close()

Epoch 1/20
Test Loss: 0.025248, Accuracy: 186061/356921 (52.13%)
Epoch 2/20
Test Loss: 0.024494, Accuracy: 190147/356921 (53.27%)
Epoch 3/20
Test Loss: 0.024182, Accuracy: 191807/356921 (53.74%)
Epoch 4/20
Test Loss: 0.023956, Accuracy: 192783/356921 (54.01%)
Epoch 5/20
Test Loss: 0.023752, Accuracy: 194155/356921 (54.40%)
Epoch 6/20
Test Loss: 0.023493, Accuracy: 195485/356921 (54.77%)
Epoch 7/20
Test Loss: 0.023405, Accuracy: 196465/356921 (55.04%)
Epoch 8/20
Test Loss: 0.023399, Accuracy: 196490/356921 (55.05%)
Epoch 9/20
Test Loss: 0.023310, Accuracy: 197214/356921 (55.25%)
Epoch 10/20
Test Loss: 0.023255, Accuracy: 197531/356921 (55.34%)
Epoch 11/20
Test Loss: 0.023247, Accuracy: 197299/356921 (55.28%)
Epoch 12/20
Test Loss: 0.023208, Accuracy: 198107/356921 (55.50%)
Epoch 13/20
Test Loss: 0.023182, Accuracy: 197617/356921 (55.37%)
Epoch 14/20
Test Loss: 0.023204, Accuracy: 197403/356921 (55.31%)
Epoch 15/20
Test Loss: 0.023167, Accuracy: 197101/356921 (55.22%)
Epoch 16/20
Test Lo